In [4]:
import os
from pathlib import Path
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
print(sys.executable)
print(sys.path)

d:\Programing\Ai_Music_Tagging\venv\Scripts\python.exe
['C:\\Users\\kz\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\kz\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\kz\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\kz\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\Programing\\Ai_Music_Tagging\\venv', '', 'd:\\Programing\\Ai_Music_Tagging\\venv\\Lib\\site-packages', 'd:\\Programing\\Ai_Music_Tagging\\venv\\Lib\\site-packages\\win32', 'd:\\Programing\\Ai_Music_Tagging\\venv\\Lib\\site-packages\\win32\\lib', 'd:\\Programing\\Ai_Music_Tagging\\venv\\Lib\\site-packages\\Pythonwin']


Configurations for Model tuning.

In [ ]:
# Paths
# adjust path if notebook is elsewhere
DATA_RAW = Path(r"D:\Programing\Ai_Music_Tagging\data\genres_original")
DATA_PROCESSED = Path("../data/processed")
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Audio params
SAMPLE_RATE = 22050
SEGMENT_DURATION = 5  # seconds
N_MELS = 128
HOP_LENGTH = 512
N_FFT = 2048

D:\Programing\data\genres_original
True


In [24]:
print(DATA_RAW)
genres = sorted([g.name for g in DATA_RAW.iterdir() if g.is_dir()])
print("Genres:", genres)

# Count files
for g in genres:
    count = len(list((DATA_RAW / g).glob("*.wav")))
    print(f"{g}: {count}")

D:\Programing\Ai_Music_Tagging\data\genres_original
Genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
blues: 100
classical: 100
country: 100
disco: 100
hiphop: 100
jazz: 100
metal: 100
pop: 100
reggae: 100
rock: 100


Preprocessing 

In [ ]:
def preprocess_file(filepath, genre):
    # Load audio
    y, sr = librosa.load(filepath, sr=SAMPLE_RATE, mono=True)

    samples_per_segment = SEGMENT_DURATION * SAMPLE_RATE
    total_segments = len(y) // samples_per_segment

    saved_files = []
    for i in range(total_segments):
        start = i * samples_per_segment
        end = start + samples_per_segment
        segment = y[start:end]

        # Compute Mel spectrogram
        mel = librosa.feature.melspectrogram(
            y=segment,
            sr=SAMPLE_RATE,
            n_fft=N_FFT,
            hop_length=HOP_LENGTH,
            n_mels=N_MELS
        )
        mel_db = librosa.power_to_db(mel, ref=np.max)

        # Normalize to [-1,1]
        mel_db = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-9)

        # Save
        out_dir = DATA_PROCESSED / genre
        out_dir.mkdir(parents=True, exist_ok=True)
        out_name = f"{filepath.stem}_seg{i}.npy"
        np.save(out_dir / out_name, mel_db)
        saved_files.append(out_dir / out_name)

    return saved_files